In [3]:
import pandas as pd
import datetime
import gzip
import io
import os
from nltk.sentiment import SentimentIntensityAnalyzer

In [4]:
google = pd.read_json(r'datasets/google/estados_seleccionados_restaurantes.json.gz')
google

,gmap_id,latitude,longitude,category,avg_rating,Estado_Perteneciente,name
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.292130,['Korean restaurant'],4.4,California,San Soo Dang
6,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,33.916402,-118.010855,['Restaurant'],4.5,California,Vons Chicken
67,0x89c6c89efcaed69d:0xded973f6033e7dba,39.940293,-75.150923,['Restaurant'],4.0,New Jersey,Vivi Bubble Tea
111,0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,26.641377,-81.940545,['Restaurant'],5.0,Florida,Cape Seafood Shack
112,0x88c2e4e34f1ed783:0x76c5da381c499d79,27.867489,-82.702971,['Buffet restaurant'],5.0,Florida,Fresh Point Country Buffet
...,...,...,...,...,...,...,...
2970302,0x88d9b3cd3f274101:0x64335c60e4e3065e,25.848295,-80.174799,['German restaurant' 'Bar' 'Restaurant'],4.3,Florida,Royal Bavarian Schnitzel Haus
2970307,0x88e6d78a11694343:0xaed512a21b391dcd,29.142263,-80.986379,['Pizza restaurant'],4.4,Florida,Panheads Pizzeria
2970335,0x88d9dd6e78851851:0x849dcf273582261c,25.506121,-80.438215,['Sandwich shop' 'Fast food restaurant' 'Takeo...,4.3,Florida,Subpreme Grill
2970336,0x89c6c4072624d2a9:0x4437b2d55e9a5811,39.923594,-75.245269,['Coffee shop' 'Bagel shop' 'Bakery' 'Breakfas...,4.0,New Jersey,Dunkin'


In [4]:
yelp = pd.read_csv(r'datasets/yelp/business.csv.gz')
yelp

,business_id,latitude,longitude,stars,categories,Estado_Perteneciente
0,Pns2l4eNsfO8kk83dixA6A,34.426679,-119.711197,5.0,"Doctors, Traditional Chinese Medicine, Naturop...",California
1,mpf3x-BjTdTEA3yCZrAYPw,38.551126,-90.335695,3.0,"Shipping Centers, Local Services, Notaries, Ma...",Illinois
2,tUFrWirKiKi_TAnsVWINQQ,32.223236,-110.880452,3.5,"Department Stores, Shopping, Fashion, Home & G...",Arizona
3,MTSW4McQd7CbVtyjqoe9mw,39.955505,-75.155564,4.0,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...",New Jersey
4,mWMc6_wTdE0EUBKIGXDVfA,40.338183,-75.471659,4.5,"Brewpubs, Breweries, Food",New Jersey
...,...,...,...,...,...,...
144666,2O2K6SXPWv56amqxCECd4w,39.856185,-75.427725,4.5,"Restaurants, Comfort Food, Food, Food Trucks, ...",New Jersey
144667,c8GjPIOTGVmIemT7j5_SyQ,36.115118,-86.766925,4.0,"Pets, Nurseries & Gardening, Pet Stores, Hobby...",Tennessee
144668,_QAMST-NrQobXduilWEqSw,39.908707,-86.065088,3.5,"Shopping, Jewelry, Piercing, Toy Stores, Beaut...",Indiana
144669,mtGm22y5c2UHNXDFAjaPNw,38.782351,-89.950558,4.0,"Fitness/Exercise Equipment, Eyewear & Optician...",Illinois


In [5]:
restaurant_counts = google['Estado_Perteneciente'].value_counts()
restaurant_counts

Estado_Perteneciente
California    28862
New Jersey    18724
Florida       13386
Illinois      10030
Name: count, dtype: int64

In [6]:
population_data = {
    'California': 39029342,
    'New Jersey': 9261699,
    'Florida': 22244823,
    'Illinois': 12582032
}

In [8]:
proportion = {state: f"{proportion[state]:.2f} restaurantes por cada 1000 habitantes" for state in proportion}
proportion

NameError: name 'proportion' is not defined

In [9]:
top_categories = google['category'].value_counts().nlargest(10)
filtered_categories = top_categories[top_categories.index.str.contains('estaur', case=False)]
filtered_categories

category
['Restaurant']                                                                                                             11868
['Mexican restaurant']                                                                                                      3247
['Sandwich shop' 'Caterer' 'Fast food restaurant' 'Takeout Restaurant'\n 'Restaurant']                                      1946
['Chinese restaurant']                                                                                                      1943
['Pizza restaurant']                                                                                                        1914
['American restaurant']                                                                                                      974
['Fast food restaurant']                                                                                                     769
['Italian restaurant']                                                                  

In [11]:
top_categories = google['category'].value_counts().nlargest(10)
filtered_categories = top_categories[top_categories.index.str.contains('estaur', case=False)].index

df_filtered = google[google['category'].isin(filtered_categories)]
df_filtered

,gmap_id,latitude,longitude,category,avg_rating,Estado_Perteneciente,name
6,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,33.916402,-118.010855,['Restaurant'],4.5,California,Vons Chicken
67,0x89c6c89efcaed69d:0xded973f6033e7dba,39.940293,-75.150923,['Restaurant'],4.0,New Jersey,Vivi Bubble Tea
111,0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,26.641377,-81.940545,['Restaurant'],5.0,Florida,Cape Seafood Shack
265,0x808f87f90c1f661f:0xf384e804a61e0c0b,37.764203,-122.214647,['Restaurant'],5.0,California,Mariscos el poblano
476,0x80dcd95d192d988b:0x68795f58e35bf888,33.748329,-117.866045,['Restaurant'],4.0,California,Off The Hoof
...,...,...,...,...,...,...,...
2970135,0x80dd2997af971b1d:0x9651e630fae0c790,33.832778,-117.957782,['Mexican restaurant'],4.3,California,Brito's 1 Mexican Food Taquería
2970182,0x88d9bf83f615ee57:0x19ab538559a91bbb,25.732026,-80.416455,['Fast food restaurant'],5.0,Florida,DAIRY QUEEN LTD BRAZIER
2970185,0x88d9b0a5df49c933:0x121c745a993502c1,25.822775,-80.271100,['Fast food restaurant'],4.3,Florida,DAIRY QUEEN STORE
2970307,0x88e6d78a11694343:0xaed512a21b391dcd,29.142263,-80.986379,['Pizza restaurant'],4.4,Florida,Panheads Pizzeria


In [13]:
df_filtered['category'].unique()

array(["['Restaurant']", "['Mexican restaurant']",
       "['Fast food restaurant']", "['Chinese restaurant']",
       "['American restaurant']",
       "['Sandwich shop' 'Caterer' 'Fast food restaurant' 'Takeout Restaurant'\n 'Restaurant']",
       "['Italian restaurant']", "['Pizza restaurant']",
       "['Coffee shop' 'Bagel shop' 'Bakery' 'Breakfast restaurant' 'Donut shop'\n 'Fast food restaurant' 'Takeout Restaurant']",
       "['Seafood restaurant']"], dtype=object)